In [1]:
import pandas as pd
df = pd.read_csv("USA_Housing.csv")

In [4]:
df.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
0,79545.45857,5.682861,7.009188,4.09,23086.80050,1.059034e+06
1,79248.64245,6.002900,6.730821,3.09,40173.07217,1.505891e+06
2,61287.06718,5.865890,8.512727,5.13,36882.15940,1.058988e+06
3,63345.24005,7.188236,5.586729,3.26,34310.24283,1.260617e+06
4,59982.19723,5.040555,7.839388,4.23,26354.10947,6.309435e+05


In [3]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold,train_test_split

In [4]:
x=df.drop("Price",axis=1).values
y=df["Price"].values.reshape(-1,1)

In [7]:
scaler=StandardScaler()
x_scaled=scaler.fit_transform(x)

In [9]:
kf=KFold(n_splits=5,shuffle=True, random_state=42)

In [11]:
best_beta=None
best_r2=-np.inf
r2_scores=[]

In [19]:
for fold, (train_idx, test_idx) in enumerate(kf.split(x_scaled)):      #kf.split(X_scaled) generates 5 splits (since we used KFold(n_splits=5)).
    X_train, X_test = x_scaled[train_idx], x_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

In [23]:
X_train_bias = np.c_[np.ones((X_train.shape[0], 1)), X_train]
X_test_bias = np.c_[np.ones((X_test.shape[0], 1)), X_test]

In [25]:
# Compute beta using Least Squares: β = (XᵀX)^(-1) Xᵀy
beta = np.linalg.inv(X_train_bias.T @ X_train_bias) @ (X_train_bias.T @ y_train)
     # Predictions
y_pred = X_test_bias @ beta


In [27]:
    # R2 score
r2 = r2_score(y_test, y_pred)
r2_scores.append(r2)
print(f"Fold {fold+1}: R2 Score = {r2:.4f}")

Fold 5: R2 Score = 0.9244


In [32]:
  # Track best beta
if r2 > best_r2:
    best_r2 = r2
    best_beta = beta

print("\nAverage R2 Score across 5 folds:", np.mean(r2_scores))
print("Best R2 Score:", best_r2)


Average R2 Score across 5 folds: 0.9243869413350316
Best R2 Score: 0.9243869413350316


In [36]:
# Step e: Train on 70% using best beta and test on 30%
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.3, random_state=42)

X_train_bias = np.c_[np.ones((X_train.shape[0], 1)), X_train]
X_test_bias = np.c_[np.ones((X_test.shape[0], 1)), X_test]

# Train using best beta on 70% data
y_train_pred = X_train_bias @ best_beta
y_test_pred = X_test_bias @ best_beta

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("\nFinal Model Performance using Best Beta:")
print(f"Train R2 Score: {train_r2:.4f}")
print(f"Test R2 Score: {test_r2:.4f}")


Final Model Performance using Best Beta:
Train R2 Score: 0.9193
Test R2 Score: 0.9147
